In [1]:
print("ok")

ok


In [51]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [52]:
PINECONE_API_KEY = "dc8acf0a-9af9-44a4-b3b2-a1a6d7caf258"
PINECONE_API_ENV = "us-east-1"

In [53]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [54]:
extracted_data = load_pdf("data/")

In [55]:
#extracted_data

In [56]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [57]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [58]:
#text_chunks

In [59]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [60]:
embeddings = download_hugging_face_embeddings()


c:\Users\Anas\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [61]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [96]:
PINECONE_API_KEY = "eac98585-8fa0-46d2-84d4-6d0d2a6e74b4"
PINECONE_API_ENV = "us-east-1"

In [98]:
import pinecone

# Initialize Pinecone with your API key and environment
PINECONE_API_KEY = "eac98585-8fa0-46d2-84d4-6d0d2a6e74b4"
PINECONE_API_ENV = "us-east-1"
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# Connect to the index
index_name = pinecone.Index("medicalbot")


In [102]:
texts = [t.page_content for t in text_chunks]
